In [ ]:
import cv2
from scipy.signal import convolve2d
import numpy as np
import os
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fft import dctn, idctn
import plotly.graph_objects as go
from tqdm import tqdm

In [ ]:
from utils.fonctions import lightDirection, lightsDirection, getI, integrationSCS, stereophotometrie

In [ ]:
images = []

path = "./data/tablette/"

for image_path in tqdm(os.listdir(path)):
    image = cv2.imread(path + image_path)
    images.append(image)

In [ ]:
images[0].shape

In [ ]:
S = lightsDirection(images, [350, 1760, 1975, 3315])

In [ ]:
S

In [ ]:
ratio = images[0].shape[1] / images[0].shape[0]
imagesR = []
imagesG = []
imagesB = []
imagesGray = []

images2 = []
for image in images:
    i = cv2.resize(image, (1000, int(1000 * ratio)))
    imagesR.append(i[:,:,0])
    imagesG.append(i[:,:,1])
    imagesB.append(i[:,:,2])
    imagesGray.append(np.mean(i, axis=2))

In [ ]:
imagesGray[0].shape

In [ ]:
masque = np.ones(imagesGray[0].shape)
exterieur = masque.flatten() == 0
exterieur= np.reshape(exterieur, masque.shape)

In [ ]:
I_r = getI(imagesR, corrige=True)
I_g = getI(imagesG, corrige=True)
I_b = getI(imagesB, corrige=True)
I_gray = getI(imagesGray, corrige=True)

In [ ]:
def stereophotometrie(I,S,masque=None):
    pseudo_inverse_S = np.linalg.pinv(S)
    m = pseudo_inverse_S @ I

    rho_estime = np.sqrt(np.sum(np.square(m), 0))
    N_estime = m / (rho_estime + 1e-3)
    N_estime[:, masque.flatten() == 0] = 0

    return rho_estime, N_estime

In [ ]:
rho_estime_r, N_estime_r = stereophotometrie(I_r, S, masque)
rho_estime_g, N_estime_g = stereophotometrie(I_g, S, masque)
rho_estime_b, N_estime_b = stereophotometrie(I_b, S, masque)
rho_estime_gray, N_estime_gray = stereophotometrie(I_gray, S, masque)

In [ ]:
N_estime_gray[2, exterieur.flatten()] = 1

In [ ]:
p_estime = np.reshape(-N_estime_gray[0, :] / N_estime_gray[2, :], masque.shape)
p_estime[exterieur] = 0
q_estime = np.reshape(-N_estime_gray[1, :] / N_estime_gray[2, :], masque.shape)
q_estime[exterieur] = 0

In [ ]:
z_estime = integrationSCS(p_estime, q_estime)

In [ ]:
nb_lignes, nb_colonnes = z_estime.shape
mid_row = np.floor(nb_lignes / 2).astype(int)
mid_col = np.floor(nb_colonnes / 2).astype(int)

# Vérification de la condition et modification du tableau z_estime si nécessaire
if z_estime[mid_row, mid_col] < z_estime[1, 1]:
    z_estime = -z_estime
z_estime[exterieur] = 0

In [ ]:
plt.imshow(z_estime, cmap='viridis')  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

plt.imshow()  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
image_ref = cv2.resize(images[0], (1000, int(1000 * ratio)))
plt.imshow(image_ref)  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
I_r_m = np.median(I_r, axis=0)
I_g_m = np.median(I_g, axis=0)
I_b_m = np.median(I_b, axis=0)

In [ ]:
I_r_m = I_r_m.reshape(masque.shape).astype(np.uint8)
I_g_m = I_g_m.reshape(masque.shape).astype(np.uint8)
I_b_m = I_b_m.reshape(masque.shape).astype(np.uint8)

rho_estime_color = np.stack((I_r_m, I_g_m, I_b_m), axis=-1)
plt.imshow((rho_estime_color).astype(np.uint8))  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
b, g, r = cv2.split(image_ref)

# Créer une figure pour afficher les canaux côte à côte
plt.figure(figsize=(10, 4))  # Définir la taille de la figure

# Afficher chaque canal séparément
plt.subplot(1, 3, 1)
plt.imshow(r, cmap='Reds')
plt.title('Canal Rouge')
plt.axis('off')  # Désactiver les axes

plt.subplot(1, 3, 2)
plt.imshow(g, cmap='Greens')
plt.title('Canal Vert')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(b, cmap='Blues')
plt.title('Canal Bleu')
plt.axis('off')

plt.tight_layout()  # Ajuster la disposition pour éviter les chevauchements
plt.show()

In [ ]:
rho_estime_r = rho_estime_r.reshape(masque.shape).astype(np.uint8)
rho_estime_g = rho_estime_g.reshape(masque.shape).astype(np.uint8)
rho_estime_b = rho_estime_b.reshape(masque.shape).astype(np.uint8)

lmd_r = 1
lmd_g = 1
lmd_b = 1

rho_estime_color = np.stack((lmd_r * rho_estime_r, lmd_g * rho_estime_g, lmd_b * rho_estime_b), axis=-1)
plt.imshow((rho_estime_color).astype(np.uint8))  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
b, g, r = cv2.split(rho_estime_color)

# Créer une figure pour afficher les canaux côte à côte
plt.figure(figsize=(10, 4))  # Définir la taille de la figure

# Afficher chaque canal séparément
plt.subplot(1, 3, 1)
plt.imshow(r, cmap='Reds')
plt.title('Canal Rouge')
plt.axis('off')  # Désactiver les axes

plt.subplot(1, 3, 2)
plt.imshow(g, cmap='Greens')
plt.title('Canal Vert')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(b, cmap='Blues')
plt.title('Canal Bleu')
plt.axis('off')

plt.tight_layout()  # Ajuster la disposition pour éviter les chevauchements
plt.show()

In [ ]:
def hex_to_rgb(hex_valeur):
    # Assurez-vous que la valeur hexadécimale commence par '#'
    if hex_valeur.startswith('#'):
        hex_valeur = hex_valeur[1:]

    # Vérification de la longueur de la chaîne hexadécimale
    if len(hex_valeur) != 6:
        raise ValueError("La valeur hexadécimale doit être de 6 caractères (sans compter '#')")

    # Extraction des composantes RVB
    rouge = int(hex_valeur[0:2], 16)
    vert = int(hex_valeur[2:4], 16)
    bleu = int(hex_valeur[4:], 16)

    return rouge, vert, bleu

In [ ]:
markers = [[3,213,73,243], [91,212,161,242], [4,252,74,281], [92,252,162,282], [5,291,74,320], [92,290,163,321], [7,330,74,359], [93,330,163,358], [6,370,74,400], [94,370,165,398], [5,410,77,437], [96,407,165,437]]
markers_colors_hex = ["#0885a1", "#343434", "#bb5695", "#555555", "#e7c71f", "#7a7a7a", "#af363c", "#a0a0a0", "#469449", "#c8c8c8", "#383d96", "#f3f3f3"]

markers_colors_hex = [hex_to_rgb(i) for i in markers_colors_hex]

print(len(markers), len(markers_colors_hex))

In [ ]:
image_ref = cv2.resize(images[0], (1000, int(1000 * ratio)))

def couleur_mediane_rectangle(img, coords):
    x1, y1, x2, y2 = coords

    # Extraire le rectangle de l'image
    rectangle = img[y1:y2, x1:x2]

    # Calculer la couleur médiane dans le rectangle
    couleur_mediane = np.median(rectangle, axis=(0, 1)).astype(int)
    return couleur_mediane

couleurs_medianes = []
# Obtenir la couleur médiane pour chaque rectangle défini
for idx, coord in enumerate(markers, start=1):
    couleur_mediane = couleur_mediane_rectangle(rho_estime_color, coord)
    couleurs_medianes.append(couleur_mediane)
    print(f"Couleur médiane du rectangle {idx}: {couleur_mediane}")
    cv2.rectangle(rho_estime_color, (coord[0], coord[1]), (coord[2], coord[3]), couleur_mediane.tolist(), thickness=2)

matrice_couleurs = np.array(couleurs_medianes)

produit_RG = np.multiply(matrice_couleurs[:, 0], matrice_couleurs[:, 1]).reshape(-1, 1)
produit_RB = np.multiply(matrice_couleurs[:, 0], matrice_couleurs[:, 2]).reshape(-1, 1)
produit_GB = np.multiply(matrice_couleurs[:, 1], matrice_couleurs[:, 2]).reshape(-1, 1)

# Ajouter une colonne de valeurs R**2, G**2 et B**2 à la matrice
valeurs_carrees = np.square(matrice_couleurs[:, :3])  # Obtenir les valeurs R**2, G**2, B**2
matrice_couleurs = np.hstack((matrice_couleurs, valeurs_carrees))
matrice_couleurs = np.hstack((matrice_couleurs, produit_RG, produit_RB, produit_GB))

matrice_real_colors = np.array(markers_colors_hex)


In [ ]:
cv2.imshow('Image avec rectangles', rho_estime_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def matrice_couleurs_rectangle(img, coords):
    x1, y1, x2, y2 = coords

    # Extraire les pixels du rectangle de l'image
    rectangle = img[y1:y2, x1:x2]

    # Reformater les pixels en une matrice de couleurs
    matrice_pixels = rectangle.reshape((-1, 3))

    return matrice_pixels

# Créer une matrice des couleurs pour chaque rectangle
matrices_couleurs_rectangles = []
matrices_couleurs_reelles = []
for idx, coord in enumerate(markers, start=1):
    matrice_couleurs_ = matrice_couleurs_rectangle(rho_estime_color, coord)
    matrices_couleurs_rectangles.append(matrice_couleurs_)
    
    couleur_reelle = np.tile(markers_colors_hex[idx - 1], (matrice_couleurs_.shape[0], 1))
    matrices_couleurs_reelles.append(couleur_reelle)

# Concaténer toutes les matrices des couleurs
matrice_couleurs_complete = np.vstack(matrices_couleurs_rectangles)
matrice_couleurs_reelles_complete = np.vstack(matrices_couleurs_reelles)

In [ ]:
produit_RG = np.multiply(matrice_couleurs_complete[:, 0], matrice_couleurs_complete[:, 1]).reshape(-1, 1)
produit_RB = np.multiply(matrice_couleurs_complete[:, 0], matrice_couleurs_complete[:, 2]).reshape(-1, 1)
produit_GB = np.multiply(matrice_couleurs_complete[:, 1], matrice_couleurs_complete[:, 2]).reshape(-1, 1)

# Ajouter une colonne de valeurs R**2, G**2 et B**2 à la matrice
valeurs_carrees = np.square(matrice_couleurs_complete[:, :3])  # Obtenir les valeurs R**2, G**2, B**2
matrice_couleurs_complete = np.hstack((matrice_couleurs_complete, valeurs_carrees))
matrice_couleurs_complete = np.hstack((matrice_couleurs_complete, produit_RG, produit_RB, produit_GB))

In [ ]:
matrice_real_colors.shape

In [ ]:
pseudo_inverse = np.linalg.pinv(matrice_couleurs)
coeff_med = pseudo_inverse @ matrice_real_colors

coeff_med.shape

In [ ]:
matrice_real_colors[:,0]

In [ ]:
matrice_couleurs @ coeff_med

In [ ]:
pseudo_inverse = np.linalg.pinv(matrice_couleurs_complete)
coeff_all = pseudo_inverse @ matrice_couleurs_reelles_complete

coeff_all.shape

In [ ]:
rho_estime_color_ = rho_estime_color.reshape((-1, 3))

produit_RG = np.multiply(rho_estime_color_[:, 0], rho_estime_color_[:, 1]).reshape(-1, 1)
produit_RB = np.multiply(rho_estime_color_[:, 0], rho_estime_color_[:, 2]).reshape(-1, 1)
produit_GB = np.multiply(rho_estime_color_[:, 1], rho_estime_color_[:, 2]).reshape(-1, 1)

valeurs_carrees = np.square(rho_estime_color_[:, :3])  # Obtenir les valeurs R**2, G**2, B**2
rho_estime_color_ = np.hstack((rho_estime_color_, valeurs_carrees))
rho_estime_color_ = np.hstack((rho_estime_color_, produit_RG, produit_RB, produit_GB))

rho_estime_color_.shape

In [ ]:
new_colors_med = rho_estime_color_ @ coeff_med
new_colors_all = rho_estime_color_ @ coeff_all
new_colors_med.shape, new_colors_all.shape

In [ ]:
new_colors_med = new_colors_med.reshape(rho_estime_color.shape).astype(np.uint8)
new_colors_all = new_colors_all.reshape(rho_estime_color.shape).astype(np.uint8)
new_colors_med.shape, new_colors_all.shape

In [ ]:
plt.imshow(new_colors_med)  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
plt.imshow(new_colors_all)  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
plt.imshow(image_ref)  # Utilisation de log pour mieux visualiser les différences
plt.colorbar()  # Ajout d'une barre de couleur pour indiquer l'échelle
plt.show()

In [ ]:
x, y = np.meshgrid(np.arange(z_estime.shape[1]), np.arange(z_estime.shape[0]))

In [ ]:
print(x.shape)
print(y.shape)
print(z_estime.shape)
print(rho_estime_color.shape)

In [ ]:
fig = go.Figure(data=[go.Surface(z=z_estime, x=x, y=y)])

# Mise en forme et affichage de la figure
fig.update_layout(title='Surface 3D', autosize=False,
                  width=800, height=600,
                  margin=dict(l=40, r=40, b=40, t=40))
fig.show()

In [ ]:
"""import plotly.graph_objs as go
import plotly.express as px
import numpy as np

# Génération de données aléatoires pour les coordonnées x, y, z
np.random.seed(42)
num_points = 100
x_data = x.flatten()
y_data = y.flatten()
z_data = z_estime.flatten()
# Génération de couleurs RVB aléatoires pour chaque point
colors_rgb = np.random.randint(0, 256, size=(x_data.shape[0], 3))

print(rho_estime_color.reshape(-1,3).shape, colors_rgb.shape)

# Conversion des valeurs RVB en chaînes hexadécimales pour Plotly
colors_hex = ['#%02x%02x%02x' % tuple(rgb) for rgb in rho_estime_color.reshape(-1,3).astype(np.uint8)]

num_points_to_display = 40000  # Nombre de points à afficher
random_indices = np.random.choice(x_data.shape[0], size=num_points_to_display, replace=False)

x_data = x_data[random_indices]
y_data = y_data[random_indices]
z_data = z_data[random_indices]
colors_subset = [colors_hex[i] for i in random_indices]

# Création de la figure 3D
fig = go.Figure(data=[go.Scatter3d(
    x=x_data,
    y=y_data,
    z=z_data,
    mode='markers',
    marker=dict(
        size=1,
        color=colors_hex,  # Utilisation des couleurs aléatoires RVB
        opacity=0.8
    )
)])

# Mise en forme du layout
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis')
    ),
    title='Points 3D avec couleurs RVB aléatoires'
)

# Affichage de la figure
fig.show()"""